In [11]:
import pandas as pd
import os

def generate_bash_script(mode):
    """
    Reads data from a CSV file and generates a bash script to apply tc rules iteratively.
    Mode can be 'Baseline' or 'Best'.
    """

    # Validate mode
    if mode not in ["Baseline", "Best"]:
        print("Error: mode must be either 'Baseline' or 'Best'")
        return

    # Define filenames based on mode
    csv_file_path = f"{mode}_Satellite_Australia_Simulation_Log_cleaned_starlink_downlink.csv"
    script_filename = f"apply_tc_rules_{mode.lower()}.sh"
    config_filename = f"router_configs_{mode.lower()}.csv"

    # Define required columns
    required_columns = [
        f'Sydney_{mode}_Thrpt',
        f'Sydney_{mode}_Latency',
        f'Sydney_{mode}_BER_QPSK',
        f'Melbourne_{mode}_Thrpt',
        f'Melbourne_{mode}_Latency',
        f'Melbourne_{mode}_BER_QPSK'
    ]

    try:
        df = pd.read_csv(csv_file_path)
        print("Dataframe Columns: ",df.columns)
    except FileNotFoundError:
        print(f"Error: File not found: '{csv_file_path}'")
        return
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    if not all(col in df.columns for col in required_columns):
        missing = [col for col in required_columns if col not in df.columns]
        print(f"Error: Missing columns: {', '.join(missing)}")
        return

    script = [
        "#!/bin/bash",
        "set -x",
        "",
        "# Load environment settings",
        "if [ ! -f ./utils/settings.sh ]; then",
        "    echo \"Error: settings.sh not found\"",
        "    exit 1",
        "fi",
        "source ./utils/settings.sh",
        ""
    ]

    router_configs = []

    for i, row in df.iterrows():
        try:
            sydney_thrpt = pd.to_numeric(row[f'Sydney_{mode}_Thrpt'], errors='coerce')
            sydney_latency = pd.to_numeric(row[f'Sydney_{mode}_Latency'], errors='coerce')
            sydney_ber_qpsk = pd.to_numeric(row[f'Sydney_{mode}_BER_QPSK'], errors='coerce')

            melbourne_thrpt = pd.to_numeric(row[f'Melbourne_{mode}_Thrpt'], errors='coerce')
            melbourne_latency = pd.to_numeric(row[f'Melbourne_{mode}_Latency'], errors='coerce')
            melbourne_ber_qpsk = pd.to_numeric(row[f'Melbourne_{mode}_BER_QPSK'], errors='coerce')

            if any(pd.isna(val) for val in [sydney_thrpt, sydney_latency, sydney_ber_qpsk,
                                            melbourne_thrpt, melbourne_latency, melbourne_ber_qpsk]):
                script.append(f"# Skipped datapoint {i + 1}: Invalid or missing data")
                continue

            thrpt = min(round(sydney_thrpt), round(melbourne_thrpt))
            latency = round(sydney_latency + melbourne_latency)
            ber_qpsk = max(sydney_ber_qpsk, melbourne_ber_qpsk)
            loss_pct = int(round(ber_qpsk * 100, 0))

            router_configs.append({
                'thrpt': thrpt,
                'latency': latency,
                'ber_qpsk': ber_qpsk,
                'loss_pct': loss_pct
            })

            script += [
                f"# --- Datapoint {i + 1} ---",
                f"echo \"Applying {thrpt} Mbps, {latency} ms, Loss: {loss_pct:.4f}%\"",
                "",
                "# Configure ens37 (delay + dualpi2)",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 root handle 1: netem delay {latency}ms loss {loss_pct}%\"",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 parent 1:1 handle 2: dualpi2\"",
                "",
                "# Configure ens38 (rate + dualpi2)",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 root handle 1: tbf rate {thrpt}mbit burst 32kbit latency 100ms\"",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 parent 1:1 handle 2: dualpi2\"",
                "",
                f"echo \"Completed setup for datapoint {i + 1}\"",
                "sleep 30",
                ""
            ]

        except Exception as e:
            script.append(f"# Skipped datapoint {i + 1}: Exception: {e}")
            continue

    # Save router config
    pd.DataFrame(router_configs).to_csv(config_filename, index=False)

    # Add cleanup section
    script += [
        "# Cleanup",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
        "echo \"All datapoints processed.\"",
        "exit 0"
    ]

    # Write the bash script
    with open(script_filename, "w") as f:
        f.write("\n".join(script))

    print(f"\n✅ Bash script saved to: {script_filename}")
    print(f"✅ Router config saved to: {config_filename}")
    print("ℹ️  Make the script executable with: chmod +x", script_filename)
    print("ℹ️  Then run it: ./", script_filename)


if __name__ == "__main__":
    # ✅ SET THIS VARIABLE: Choose "Baseline" or "Best"
    mode = "Baseline"
    generate_bash_script(mode)

    mode = "Best"
    generate_bash_script(mode)


Dataframe Columns:  Index(['Time', 'Sydney_Baseline_SAT_ID', 'Sydney_Baseline_SNR',
       'Sydney_Baseline_RSSI', 'Sydney_Baseline_Thrpt',
       'Sydney_Baseline_BER_MQAM', 'Sydney_Baseline_BER_QPSK',
       'Sydney_Baseline_Latency', 'Melbourne_Baseline_SAT_ID',
       'Melbourne_Baseline_SNR', 'Melbourne_Baseline_RSSI',
       'Melbourne_Baseline_Thrpt', 'Melbourne_Baseline_BER_MQAM',
       'Melbourne_Baseline_BER_QPSK', 'Melbourne_Baseline_Latency'],
      dtype='object')

✅ Bash script saved to: apply_tc_rules_baseline.sh
✅ Router config saved to: router_configs_baseline.csv
ℹ️  Make the script executable with: chmod +x apply_tc_rules_baseline.sh
ℹ️  Then run it: ./ apply_tc_rules_baseline.sh
Dataframe Columns:  Index(['Time', 'Sydney_Best_SAT_ID', 'Sydney_Best_SNR', 'Sydney_Best_RSSI',
       'Sydney_Best_Thrpt', 'Sydney_Best_BER_MQAM', 'Sydney_Best_BER_QPSK',
       'Sydney_Best_Latency', 'Melbourne_Best_SAT_ID', 'Melbourne_Best_SNR',
       'Melbourne_Best_RSSI', 'Melbourne_B